In [ ]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import BPR, SLRC, NBRKNN
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Train Repurchase Module with base intensity with best hyperparams (see ./testing_slrc.ipynb):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 7358/7358 [00:11<00:00, 638.75it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4386.31it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 2830.84it/s]


In [ ]:
slrc_best_params = {'emb_size': 64, 'batch_size': 64, 'lr': 0.0006142297613045982, 'l2_reg_coef': 0.0047331742711911855}

params = {
    "model": SLRC(
        base_model_class=BPR,
        base_model_config={
            "emb_size": slrc_best_params["emb_size"],
            "user_num": corpus.n_users,
            "item_num": corpus.n_items,
            "click_num": corpus.n_clicks
        },
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.train()

Epoch 1:


Batch loss = 0.648382: 100%|██████████| 4445/4445 [00:46<00:00, 95.77it/s]


Evaluation (dev):



100%|██████████| 7357/7357 [01:18<00:00, 94.15it/s] 


 {'precision': 0.05296996058175887, 'recall': 0.11711213174337326, 'ndcg': 0.10312238567113029}
Epoch 2:


...


Batch loss = 0.407367: 100%|██████████| 4445/4445 [00:44<00:00, 99.59it/s] 


Evaluation (dev):



100%|██████████| 7357/7357 [01:17<00:00, 95.38it/s] 


 {'precision': 0.05601467989669703, 'recall': 0.12955432394369182, 'ndcg': 0.10851308820929444}


SLRC(
  (base_model): BPR(
    (user_emb): Embedding(7358, 64)
    (item_emb): Embedding(11202, 64)
  )
)

Save users' embeddings for validation step (user's embedding for validation step is his vector of recommendation scores for all items at the time of validation):

In [ ]:
dev_user_emb = trainer.get_predictions(mode="dev")

100%|██████████| 7357/7357 [01:27<00:00, 84.54it/s] 


Tune Neighborhood Module hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:11<00:00, 645.05it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4430.63it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 3869.20it/s]


In [ ]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-23 17:16:00,621] A new study created in memory with name: no-name-81d739ff-b480-460a-bfa0-1ef674af2e74
100%|██████████| 7357/7357 [06:17<00:00, 19.47it/s]
[I 2023-04-23 17:22:23,884] Trial 0 finished with value: 0.0653908618873418 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.0653908618873418.
100%|██████████| 7357/7357 [06:11<00:00, 19.79it/s]
[I 2023-04-23 17:28:39,754] Trial 1 finished with value: 0.11081685513222486 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.11081685513222486.
100%|██████████| 7357/7357 [06:46<00:00, 18.10it/s]
[I 2023-04-23 17:35:31,426] Trial 2 finished with value: 0.09549438245097476 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 1 with value: 0.11081685513222486.
100%|██████████| 7357/7357 [06:38<00:00, 18.45it/s]
[I 2023-04-23 17:42:14,985] Trial 3 finished with value: 0.11056909102577692 and parameters: {'nearest_neighbors_nu

Test TAIW (calculate scores for different seeds):

In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=20,
        topk=10,
        early_stop_num=3
    )

    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": slrc_best_params["emb_size"],
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": slrc_best_params["batch_size"],
        "lr": slrc_best_params["lr"],
        "l2_reg_coef": slrc_best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    dev_user_emb = trainer.get_predictions(mode="dev")
    test_user_emb = trainer.get_predictions(mode="test")

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=None,
        topk=10,
        early_stop_num=None
    )

    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
            alpha=study.best_params["alpha"],
            user_emb=dev_user_emb
        )
    }
    params["model"].set_emb(test_user_emb)

    trainer.init_hyperparams(**params)

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])


___SEED___0
train dataset preparing...


100%|██████████| 7358/7358 [00:10<00:00, 679.69it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 2826.54it/s]


test dataset preparing...


...

dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4902.25it/s]


test dataset preparing...


100%|██████████| 7357/7357 [05:02<00:00, 24.28it/s]

{'precision': [0.06450999048525215, 0.06457795296996058, 0.06449639798831046, 0.06414299306782656, 0.06436047301889357], 'recall': [0.15753115753591798, 0.15716937136322623, 0.15741345312343014, 0.1542435238541085, 0.15622437697356328], 'ndcg': [0.12668185595179357, 0.12680662331986717, 0.12738603062291842, 0.12652970898499827, 0.12634257404156424]}


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.06441756150604867,
 'recall': 0.15651637657004921,
 'ndcg': 0.12674935858422834}

# TaoBao

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Train Repurchase Module with base intensity with best hyperparams (see ./testing_slrc.ipynb):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

In [ ]:
slrc_best_params = {'emb_size': 32, 'batch_size': 256, 'lr': 1.0851391597925009e-05, 'l2_reg_coef': 0.03239377807560215}

params = {
    "model": SLRC(
        base_model_class=BPR,
        base_model_config={
            "emb_size": slrc_best_params["emb_size"],
            "user_num": corpus.n_users,
            "item_num": corpus.n_items,
            "click_num": corpus.n_clicks
        },
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.train()

Epoch 1:


Batch loss = 0.671828: 100%|██████████| 191/191 [00:03<00:00, 56.06it/s]



Evaluation (dev):


100%|██████████| 9307/9307 [02:55<00:00, 52.90it/s]


 {'precision': 0.010443752014612655, 'recall': 0.09904193975860463, 'ndcg': 0.07283291962948076}


...


Batch loss = 0.671486: 100%|██████████| 191/191 [00:03<00:00, 56.03it/s]



Evaluation (dev):


100%|██████████| 9307/9307 [02:54<00:00, 53.33it/s]


 {'precision': 0.010411518212098422, 'recall': 0.09877332473765266, 'ndcg': 0.0728146605521322}


SLRC(
  (base_model): BPR(
    (user_emb): Embedding(10092, 32)
    (item_emb): Embedding(22286, 32)
  )
)

Save users' embeddings for validation step (user's embedding for validation step is his vector of recommendation scores for all items at the time of validation):

In [ ]:
dev_user_emb = trainer.get_predictions(mode="dev")

100%|██████████| 9307/9307 [02:51<00:00, 54.38it/s]


Tune Neighborhood Module hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:45<00:00, 222.03it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 25642.53it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 25731.91it/s]


In [ ]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    trainer.model.reset()
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-07 01:09:54,094] A new study created in memory with name: no-name-2bc2d12f-03e3-4f60-b3f9-ab72172b24ef
[I 2023-05-07 01:10:16,001] Trial 0 finished with value: 0.05490192885797611 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.05490192885797611.
[I 2023-05-07 01:10:30,038] Trial 1 finished with value: 0.07384532786701492 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.07384532786701492.
[I 2023-05-07 01:10:43,949] Trial 2 finished with value: 0.07189376061244337 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 1 with value: 0.07384532786701492.
[I 2023-05-07 01:10:57,906] Trial 3 finished with value: 0.07372222189684938 and parameters: {'nearest_neighbors_num': 40, 'alpha': 0.75}. Best is trial 1 with value: 0.07384532786701492.
[I 2023-05-07 01:11:12,634] Trial 4 finished with value: 0.06980540447170526 and parameters: {'nearest_neighbors_num': 34, 'alpha': 0

Test TAIW (calculate scores for different seeds):

In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(2):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=20,
        topk=10,
        early_stop_num=3
    )

    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": slrc_best_params["emb_size"],
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": slrc_best_params["batch_size"],
        "lr": slrc_best_params["lr"],
        "l2_reg_coef": slrc_best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    dev_user_emb = trainer.get_predictions(mode="dev")
    test_user_emb = trainer.get_predictions(mode="test")

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=None,
        topk=10,
        early_stop_num=None
    )

    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
            alpha=study.best_params["alpha"],
            user_emb=dev_user_emb
        )
    }
    params["model"].set_emb(test_user_emb)

    trainer.init_hyperparams(**params)

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])
    print(test_metrics)

In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.012173632749543355,
 'recall': 0.1177160560151857,
 'ndcg': 0.08150091049564787}

# Dunnhumby

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 2500, #items = 92339, #clicks = 2595370 (#illegal records = 0)
After preprocessing: #users = 2358, #items = 26756, #clicks = 1976796
Saving dataset in ./data//data_dunnhumby/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Train Repurchase Module with base intensity with best hyperparams (see ./testing_slrc.ipynb):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 2358/2358 [00:10<00:00, 220.61it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:10<00:00, 219.72it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 199.37it/s]


In [ ]:
slrc_best_params = {'emb_size': 32, 'batch_size': 256, 'lr': 0.0006366285017414498, 'l2_reg_coef': 0.09063752099202302}

params = {
    "model": SLRC(
        base_model_class=BPR,
        base_model_config={
            "emb_size": slrc_best_params["emb_size"],
            "user_num": corpus.n_users,
            "item_num": corpus.n_items,
            "click_num": corpus.n_clicks
        },
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.train()

Epoch 1:


Batch loss = 0.575686: 100%|██████████| 7530/7530 [02:11<00:00, 57.43it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [01:15<00:00, 31.39it/s]


 {'precision': 0.11904963937208317, 'recall': 0.182277700424532, 'ndcg': 0.1694145228041231}
Epoch 2:


...


Batch loss = 0.459155: 100%|██████████| 7530/7530 [02:16<00:00, 55.09it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [01:12<00:00, 32.59it/s]


 {'precision': 0.11646160373355961, 'recall': 0.1801680333166432, 'ndcg': 0.16614035622344148}


SLRC(
  (base_model): BPR(
    (user_emb): Embedding(2358, 32)
    (item_emb): Embedding(26756, 32)
  )
)

Save users' embeddings for validation step (user's embedding for validation step is his vector of recommendation scores for all items at the time of validation):

In [ ]:
dev_user_emb = trainer.get_predictions(mode="dev")

100%|██████████| 2357/2357 [01:14<00:00, 31.70it/s]


Tune Neighborhood Module hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:10<00:00, 221.38it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:10<00:00, 223.89it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:12<00:00, 195.26it/s]


In [ ]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-23 09:44:10,283] A new study created in memory with name: no-name-dab2f14e-1955-4c2c-a7e3-0be41ae025c0
100%|██████████| 2357/2357 [03:09<00:00, 12.42it/s]
[I 2023-04-23 09:47:25,259] Trial 0 finished with value: 0.12505135057878455 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.12505135057878455.
100%|██████████| 2357/2357 [02:48<00:00, 13.98it/s]
[I 2023-04-23 09:50:17,249] Trial 1 finished with value: 0.17325260973753215 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.17325260973753215.
100%|██████████| 2357/2357 [02:45<00:00, 14.27it/s]
[I 2023-04-23 09:53:05,837] Trial 2 finished with value: 0.16500951841871095 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 1 with value: 0.17325260973753215.
100%|██████████| 2357/2357 [02:35<00:00, 15.13it/s]
[I 2023-04-23 09:55:47,644] Trial 3 finished with value: 0.17335123850649198 and parameters: {'nearest_neighbors_

Test TAIW (calculate scores for different seeds):

In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(10):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=20,
        topk=10,
        early_stop_num=3
    )

    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": slrc_best_params["emb_size"],
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": slrc_best_params["batch_size"],
        "lr": slrc_best_params["lr"],
        "l2_reg_coef": slrc_best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    dev_user_emb = trainer.get_predictions(mode="dev")
    test_user_emb = trainer.get_predictions(mode="test")

    trainer = NBRTrainer(
        corpus=corpus,
        max_epochs=None,
        topk=10,
        early_stop_num=None
    )

    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
            alpha=study.best_params["alpha"],
            user_emb=dev_user_emb
        )
    }
    params["model"].set_emb(test_user_emb)

    trainer.init_hyperparams(**params)

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])


___SEED___5
train dataset preparing...


100%|██████████| 2358/2358 [00:10<00:00, 230.36it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 198.26it/s]


test dataset preparing...


...

100%|██████████| 2358/2358 [00:10<00:00, 232.09it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 208.60it/s]


test dataset preparing...


100%|██████████| 2357/2357 [02:00<00:00, 19.54it/s]


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.12135341535850656,
 'recall': 0.17906825951192068,
 'ndcg': 0.17062814846815227}